In [ ]:
from systemds.context import SystemDSContext
from systemds.examples.tutorials.adult import DataManager
from systemds.operator.algorithm import multiLogReg
from systemds.operator.algorithm import multiLogRegPredict
from systemds.operator.algorithm import confusionMatrix

with SystemDSContext() as sds:
    d = DataManager()

    # limit the sample size
    train_count = 15000
    test_count = 5000

    # Get train and test datasets.
    X_frame, Y_frame, Xt_frame, Yt_frame = d.get_preprocessed_dataset(sds)

    # Transformation specification
    jspec_data = d.get_jspec(sds)
    jspec_labels = sds.scalar(f'"{ {"recode": ["income"]} }"')

    # Transform frames to matrices.
    X, M1 = X_frame.transform_encode(spec=jspec_data)
    Xt = Xt_frame.transform_apply(spec=jspec_data, meta=M1)
    Y, M2 = Y_frame.transform_encode(spec=jspec_labels)
    Yt = Yt_frame.transform_apply(spec=jspec_labels, meta=M2)

    # Subsample to make training faster
    X = X[0:train_count]
    Y = Y[0:train_count]
    Xt = Xt[0:test_count]
    Yt = Yt[0:test_count]
    print(X.nCol)
    
    # Train model
    betas = multiLogReg(X, Y, verbose=False)

    # Apply model
    [_, y_pred, acc] = multiLogRegPredict(Xt, betas, Y=Yt)
    print(y_pred)
    # Confusion Matrix
    confusion_matrix_abs, _ = confusionMatrix(y_pred, Yt).compute()

    import logging
    logging.info("Confusion Matrix")
    print(confusion_matrix_abs)